# Loading data into the Gold zone from the Silver zone

![image-alt-text](https://learn.microsoft.com/en-us/fabric/onelake/media/onelake-medallion-lakehouse-architecture/onelake-medallion-lakehouse-architecture-example.png)

## First approach to loading an aggregate table in PySpark

In [ ]:
df_sales = spark.read.table("lakehouse_gold.sales") 
df_stockitem = spark.read.table("lakehouse_gold.stockitem")

In [ ]:
sales_stockitem = (
    df_sales.alias("sales")
    .join(
        df_stockitem.alias("stockitem"),
        df_sales.StockItemKey == df_stockitem.StockItemKey,
        "inner",
    )
    .select(
        "stockitem.Color",
        "stockitem.Brand",
        "sales.Quantity",
        "sales.TaxAmount",
    )
    .groupBy("stockitem.Color", "stockitem.Brand")
    .sum("sales.TaxAmount", "sales.Quantity")
    .withColumnRenamed("sum(TaxAmount)", "SumTaxAmount")
    .withColumnRenamed("sum(Quantity)", "SumQuantity")
)

sales_stockitem.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/agg_sales_stockitem")

In [ ]:
%%sql
SELECT * FROM lakehouse_gold.agg_sales_stockitem LIMIT 10

## Second approach to loading an aggregate table in SQL

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW view_agg_sales_employee
AS
SELECT
	employee.PreferredName
	, employee.EmployeeName
	, SUM(sales.TaxAmount) AS SumTaxAmount
	, SUM(sales.Quantity) AS SumQuantity
FROM lakehouse_gold.sales
	INNER JOIN lakehouse_gold.employee ON sales.SalespersonKey = Employee.EmployeeKey
GROUP BY employee.PreferredName, employee.EmployeeName

In [ ]:
view_agg_sales_employee = spark.sql("SELECT * FROM view_agg_sales_employee")
view_agg_sales_employee.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/agg_sales_employee")

In [ ]:
%%sql
SELECT * FROM lakehouse_gold.agg_sales_employee LIMIT 10